## Let's crawl

In [1]:
%pip install undetected-chromedriver selenium

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\phuon\anaconda3\lib\site-packages\torchlight-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [ ]:
# Crawl alonhadat.com.vn using undetected-chromedriver from page 2 to 500

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import json
import os
import random

def random_sleep(min_seconds, max_seconds):
    time.sleep(random.uniform(min_seconds, max_seconds))

def crawl_alonhadat_selenium(page_num, driver):
    url = f"https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/1/ha-noi/trang--{page_num}.html"
    driver.get(url)
    random_sleep(2, 4)  # Wait for page to load

    items = driver.find_elements(By.CSS_SELECTOR, "div.content-item")
    results = []
    for item in items:
        try:
            title_tag = item.find_element(By.CSS_SELECTOR, "div.ct_title a")
            title = title_tag.text.strip()
            link = "https://alonhadat.com.vn" + title_tag.get_attribute("href")

            date = item.find_element(By.CSS_SELECTOR, "div.ct_date").text.strip()

            try:
                area = item.find_element(By.CSS_SELECTOR, "div.ct_dt").text.strip().replace("Diện tích:", "")
            except NoSuchElementException:
                area = ""

            try:
                price = item.find_element(By.CSS_SELECTOR, "div.ct_price").text.strip().replace("Giá:", "")
            except NoSuchElementException:
                price = ""

            try:
                floors = item.find_element(By.CSS_SELECTOR, "span.floors").get_attribute("title")
            except NoSuchElementException:
                floors = ""

            try:
                bedrooms = item.find_element(By.CSS_SELECTOR, "span.bedroom").get_attribute("title")
            except NoSuchElementException:
                bedrooms = ""

            try:
                address = item.find_element(By.CSS_SELECTOR, "div.ct_dis").text.strip()
            except NoSuchElementException:
                address = ""

            results.append({
                "title": title,
                "url": link,
                "date": date,
                "area": area,
                "price": price,
                "floors": floors,
                "bedrooms": bedrooms,
                "address": address,
            })
        except Exception as e:
            print(f"Error parsing item: {e}")
    return results

def is_captcha_present(driver):
    try:
        # Check for the instruction text or the captcha input box
        body_text = driver.find_element(By.TAG_NAME, "body").text
        if "Nhập tên 3 con vật" in body_text or "Tôi không phải người máy" in body_text:
            return True
        # Or check for the captcha input box by id
        if driver.find_elements(By.ID, "captcha"):
            return True
    except Exception:
        pass
    return False

# --- Setup undetected-chromedriver ---
chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
driver = uc.Chrome(options=chrome_options, version_main=136)

# --- Crawl pages and save as JSONL ---
output_folder = "../Datasets/alonhadat.com/json_new"
os.makedirs(output_folder, exist_ok=True)
start_page = 401
end_page = 500

for page_num in range(start_page, end_page + 1):
    print(f"Crawling page {page_num}...")

    # CAPTCHA check and manual intervention
    while is_captcha_present(driver):
        print("\n[CAPTCHA detected!]")
        print("Please solve the CAPTCHA in the browser window, then press Enter here to continue...")
        input()
        driver.refresh()
        random_sleep(2, 4)
    
    data = crawl_alonhadat_selenium(page_num, driver)
    if data:
        output_file = os.path.join(output_folder, f"page_{page_num}.jsonl")
        with open(output_file, "w", encoding="utf-8") as file:
            for d in data:
                file.write(json.dumps(d, ensure_ascii=False) + "\n")
        print(f"Saved {len(data)} items to {output_file}")
    else:
        print(f"No data found for page {page_num}")
    random_sleep(1, 3)  # Be polite to the server

driver.quit()

Crawling page 351...


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=136.0.7103.114)
Stacktrace:
	GetHandleVerifier [0x0113FC83+61635]
	GetHandleVerifier [0x0113FCC4+61700]
	(No symbol) [0x00F605D3]
	(No symbol) [0x00F4FE20]
	(No symbol) [0x00F6DD1F]
	(No symbol) [0x00FD3E8C]
	(No symbol) [0x00FEDF19]
	(No symbol) [0x00FCD096]
	(No symbol) [0x00F9C840]
	(No symbol) [0x00F9D6A4]
	GetHandleVerifier [0x013C45A3+2701795]
	GetHandleVerifier [0x013BFD26+2683238]
	GetHandleVerifier [0x013DAA6E+2793134]
	GetHandleVerifier [0x01156945+155013]
	GetHandleVerifier [0x0115D02D+181357]
	GetHandleVerifier [0x011474D8+92440]
	GetHandleVerifier [0x01147680+92864]
	GetHandleVerifier [0x01132070+5296]
	BaseThreadInitThunk [0x75577BA9+25]
	RtlInitializeExceptionChain [0x77AEC36B+107]
	RtlClearBits [0x77AEC2EF+191]
